In [ ]:
import pandas as pd
import os
from collections import Counter
from lxml import etree
from tqdm import tqdm_notebook, tqdm
import gensim
from gensim.models import Word2Vec
import re
import xml.etree.cElementTree as ET
from pytorch_pretrained_bert import BertTokenizer, BertModel, BasicTokenizer
from xml.dom import minidom
import sklearn
from sklearn import metrics
import operator
import pymorphy2
import json
import copy
import nltk
import pickle
morph = pymorphy2.MorphAnalyzer()

## Collecting samples with candidates from the corpus

In [ ]:
mapping_to_sense_definitions = {
    
'N29241':'ЗАМОК_СТРОЕНИЕ',
'N24173':'ЗАМОК_ЗАПОР',
'N12915':'ЛУК_ОРУЖИЕ',
'N41975':'ЛУК_ОВОЩ',
'N31212': 'БОР_ЭЛЕМЕНТ',
'N13050': 'БОР_ЛЕС',
'N17912':'ДАР_ВРОЖДЕННЫЙ',
'N31026':'ДАР_ПОДАРОК',
'N27057': 'ДВИГАТЕЛЬ_АГРЕГАТ',
'N16946': 'ДВИГАТЕЛЬ_ДВИЖУЩ_СИЛА',
'N20410': 'ДЕДУШКА_СТАРИК',
'N35355': 'ДЕДУШКА_РОДСТВЕНН',
'N13903': 'ДЕКРЕТ_ОТПУСК',
'N34626': 'ДЕКРЕТ_ДОКУМЕНТ',
'N40123': 'ДЕРЕВО_ДЕРЕВЦЕ',
'N33095': 'ДЕРЕВО_ДРЕВЕСИНА',
'N34886': 'ДИАЛОГ_БЕСЕДА',
'N38215': 'ДИАЛОГ_МЖД_СТОРОНАМИ',
'N35864': 'ДИПЛОМ_ВУЗА',
'N40323': 'ДИПЛОМ_ПАМЯТНЫЙ',
'N20982': 'ДИПЛОМ_РАБОТА',
'N28236': 'ДОКТОР_НАУК',
'N30160': 'ДОКТОР_ВРАЧ',
'N25033': 'ДОЛЯ_УЧАСТЬ',
'N25297': 'ДОЛЯ_ЧАСТЬ',
'N38801': 'ДОСТИЖЕНИЕ_ЦЕЛИ',
'N34449': 'ДОСТИЖЕНИЕ_УРОВНЯ',
'N33820': 'ЖЕСТОКОСТЬ_БЕСПОЩАДНОСТЬ',
'N41393': 'ЖЕСТОКОСТЬ_ОБРАЩЕНИЕ',
'N21712': 'ЖРЕБИЙ_РЕШЕНИЕ',
'N25033': 'ЖРЕБИЙ_СУДЬБА',
'N18425': 'ЗАТЕЯ_ЗАБАВА',
'N20053': 'ЗАТЕЯ_НАЧИНАНИЕ',
'N25942': 'ЗАСТОЙ_ЗАСТОЙН_ЯВЛЕНИЕ',
'N37078': 'ЗАСТОЙ_СТАГНАЦИЯ_РАЗВИТ',
'N39421': 'ЗАТИШЬЕ_СНИЖ_АКТИВНОСТИ',
'N12860': 'ЗАТИШЬЕ_БЕЗВЕТР_ТИШЬ',
'N19420': 'ЗАТМЕНИЕ_ОДУРЕНИЕ',
'N36219': 'ЗАТМЕНИЕ_СВЕТИЛА',
    'N13120':'КАПОТ_ОДЕЖДА',
    'N15899': 'КАПОТ_МАШИНЫ',
    'N14586':'ТАЗ_КОСТЬ',
    'N30033': 'ТАЗ_ПОСУДА',
    'N15152':'СЛОГ_ЗВУК', 
    'N21947': 'СЛОГ_СТИЛЬ',
    'N16141': 'БАЙКА_ЛОЖЬ',
    'N39858': 'БАЙКА_ТКАНЬ',
    'N19345': 'ГУСЕНИЦА_МЕХАНИЗМ',
    'N21860': 'ГУСЕНИЦА_ЛИЧИНКА',
    'N25286': 'СТОПКА_КУЧА',
    'N26126': 'СТОПКА_ПОСУДА',
    'N26662': 'ГВОЗДИКА_ПРИПРАВА',
    'N31219': 'ГВОЗДИКА_РАСТЕНИЕ',
    'N28683': 'КРОНА_ДЕНЬГИ',
    'N30465': 'КРОНА_ДЕНЬГИ',
    'N33001': 'КРОНА_ДЕНЬГИ',
    'N37840': 'КРОНА_ДЕРЕВА',
    'N29853': 'АКЦИЯ_КОМПАНИИ',
    'N41588': 'АКЦИЯ_ДЕЙСТВИЕ',
    'N35039': 'ТАКСА_СОБАКА',
    'N36673': 'ТАКСА_ОПЛАТА',
    'N36575': 'РОК_МУЗЫКА',
    'N40621': 'РОК_СУДЬБА',
    
}

In [ ]:
"""
Mapping from polysemous word to its relatives
"""

csv_dir = r''
tuples = {}

for file in os.listdir(csv_dir):
    
    if file == 'relatives_доля_N25033.csv':
        
        relat_name = 'ДОЛЯ_УЧАСТЬ'
        
    else:
    
        relat_name = file.replace('relatives_', '').replace('.csv', '')
        relat_name = mapping_to_sense_definitions[relat_name.split('_')[1]]
    
    df = pd.read_csv(csv_dir+file, sep=';')
    
    
    if df.empty == True:
        print(relat_name)
        continue

    tuples[relat_name] = {}
    for i in tqdm_notebook(range(len(df))):
        tuples[relat_name][df.word.iloc[i]] = df.weight_sum.iloc[i]
    
    

In [ ]:
path_toa_processed_corpus = r'C:\Users\Taiga_1billion\Taiga_1billion\proza_ru_split_processed_multiwords_proper_hash_09_03.txt'
with open(path_to_processed_corpus, 'r', encoding='utf8') as f:
    processed_corpus = f.read()

In [ ]:
path_to_contexts_wout_replacements = r'C:\Users\Toloka_contexts_to_replacement_taiga_proza_ru\\'

for key, val in tqdm(tuples.items()):
    
    
    
    for key_2 in tqdm_notebook(val.keys()):
        
        target = key_2
        filename = key+'_'+key_2+'.txt'

        if os.path.isfile(path_to_contexts_wout_replacements+filename):
            continue

        else:

            with open(path_to_contexts_wout_replacements + filename, 
                      'w', encoding='utf8')as f:
                for text in re.findall(r'((\n[а-яё# -]+\s|\n)'+target+'(\s[а-яё# -]+\n|\n))', processed_corpus):
                    f.write(text[0])

## Creating Corpus with Replaced Monosemous Relatives

In [ ]:
"""
Making Percent Weights
"""
for poly_w, relatives in tuples.items():
    
    tuples[poly_w] = {}
    
    all_weights = []
    
    for mono_w, weight in relatives.items():
        
        all_weights.append(weight)
    
    sum_weight = sum(all_weights)
    
    for mono_w, weight in relatives.items():
        
        rounded_weight = round(weight/sum_weight, 3)
        if rounded_weight == 0:
            rounded_weight = 0.001
        
        tuples[poly_w][mono_w] = rounded_weight

## Balanced Approach

In [ ]:
length_total = []
path_to_corpus_with_replaced_relats = r'C:\Users\Taiga_proza_ru_replaced_contexts\\'

desired_number_examples = 1300

for key, val in tuples.items():
    
    print(key)
    
    corpus = []
    total_len = 0
    
    poly_word = key.split('_')[0].lower()
    
    x = tuples[key]
    sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse=True)
    
    for tup in sorted_x:
        
        candidate = tup[0]
        proportion = tup[1]
        num_ex = int(round(proportion * desired_number_examples))
        if num_ex == 0:
            num_ex = 1
        print(candidate)
        
        if os.path.isfile(path_to_contexts_wout_replacements + key + '_' + candidate + '.txt'):
            with open(path_to_contexts_wout_replacements + key + '_' + candidate + '.txt', 'r', encoding='utf8') as f:
                corpus_ex = f.read()
            corpus_ex = corpus_ex.split('\n\n')

            corpus_ex_clean = list(set([re.sub(r'\b'+candidate+r'\b', '_'+ poly_word+ '_', i) for i in corpus_ex if i!='\n']))

            corpus_ex_clean = [i.replace('#', ' ').replace('-', ' ').replace('\n', ' ') for i in corpus_ex_clean][:num_ex+1]

            corpus_len_first = len(corpus_ex_clean)
            print(corpus_len_first)

            if total_len+corpus_len_first>desired_number_examples+10:

                finish = desired_number_examples - total_len
                corpus.extend(corpus_ex_clean[:finish])
                corpus_len = len(corpus_ex_clean[:finish])

            else:
                corpus.extend(corpus_ex_clean)
                corpus_len = len(corpus_ex_clean)
                
            total_len+=corpus_len
        else:
            print('MISSING', key+'_'+candidate)

    with open(path_to_corpus_with_replaced_relats + key + '.txt', 'w', encoding='utf8') as f:
        for sent in corpus:
            f.write(sent+'\n')
            
    length_total.append(key+'     '+str(len(corpus)))
    print('---TOTAL_EXAMPLES---', '\t', len(corpus), '\n')

###  Creating an xml-file with train and test sets (because the source code for training requires this type of input)

In [ ]:
#SET THIS PARAM
root_dir = path_to_corpus_with_replaced_relats

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)


root = ET.Element("corpus")
doc = ET.SubElement(root, "document")
par = ET.SubElement(doc, "paragraph")
sent = ET.SubElement(par, "sentence")


for poly_w in os.listdir(root_dir):
    
    with open(root_dir+'\\'+poly_w, 'r', encoding='utf8') as f:
        text_corpus = f.readlines()
    
    for sentence in tqdm_notebook(text_corpus):
        tokens = sentence.split()
        
        _bert_tokens = []
        
        for tok in tokens:
            _tokens = tokenizer.tokenize(tok.strip('_'))
            _bert_tokens.extend(_tokens)
            
        if len(_bert_tokens)<=509:
        
            for i in tokens:
                if '_' not in i:
                    word = ET.SubElement(sent, 'word')
                    word.set('surface_form', i)
                    word.set('lemma', i)
                else:
                    word = ET.SubElement(sent, 'word')
                    word.set('surface_form', i.strip('_'))
                    word.set('lemma', i.strip('_'))
                    word.set('wn30_key', str(sense_map[poly_w]))
            sent = ET.SubElement(par, "sentence")
        
        else:
            
            num_bert_toks = 0

            prev_bert_toks = []
            follow_bert_toks = []
            main_word_index = ''
            stop_idx_back = None
            stop_idx_forw = None

            final_tokens = []
            
            for num, el in enumerate(tokens):
                
                if '_' in el:
                    
                    main_word_index = num
                    
                    break
                    
            num_bert_toks = len(tokenizer.tokenize(tokens[main_word_index]))
                    
            if main_word_index>150:
                prev_bert_toks = tokens[main_word_index-151:main_word_index]
            else:
                prev_bert_toks = tokens[0:main_word_index]
                
            if len(tokens)>main_word_index+150:
                follow_bert_toks = tokens[main_word_index+1:main_word_index+151]
            else:
                follow_bert_toks = tokens[main_word_index+1:]
                
            for num, el in reversed(list(enumerate(prev_bert_toks))):
                if len(tokenizer.tokenize(el))+num_bert_toks>=509:
                    stop_idx_back = num
                    break
                num_bert_toks+=len(tokenizer.tokenize(el))
                                                    
            if stop_idx_back==None:
                for num, el in enumerate(follow_bert_toks):
                    if len(tokenizer.tokenize(el))+num_bert_toks>=509:
                        stop_idx_forw = num
                        break
                    num_bert_toks+=len(tokenizer.tokenize(el))
                                    
            if stop_idx_back!=None:
                final_tokens.extend(prev_bert_toks[stop_idx_back+1:]+[tokens[main_word_index]])
            elif stop_idx_forw!= None:
                final_tokens.extend(prev_bert_toks+[tokens[main_word_index]]+follow_bert_toks[:stop_idx_forw])
            else:
                final_tokens.extend(prev_bert_toks+[tokens[main_word_index]]+follow_bert_toks)

                
                                    
            for i in final_tokens:
                if '_' not in i:
                    word = ET.SubElement(sent, 'word')
                    word.set('surface_form', i)
                    word.set('lemma', i)
                else:
                    word = ET.SubElement(sent, 'word')
                    word.set('surface_form', i.strip('_'))
                    word.set('lemma', i.strip('_'))
                    word.set('wn30_key', str(sense_map[poly_w]))
            sent = ET.SubElement(par, "sentence")
                
                
tree = ET.ElementTree(root)

In [ ]:
path_to_save_train_xml_file = r"C:\Users\Taiga_1billion\Taiga_1billion\train_taiga_proza_ru.xml"
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
with open(path_to_save_train_xml_file, "wb") as f:
    f.write(xmlstr.encode('utf-8'))

# Collecting Test Dataset

In [ ]:
def sent_cleaning(sent):
    brief_cleaning = re.sub("[^-А-Яа-яёA-Za-z-]+", ' ', sent)
    tokens = [w.lower() for w in brief_cleaning.split() if w.lower() not in stopwords_rus]     
    txt = [morph.parse(token.lower().strip())[0].normal_form.strip(' ') for token in tokens]
    return txt

In [ ]:
dictionary_map_russe = {'замок_1': '0',
 'замок_2': '1',
 'лук_1': '2',
 'лук_2': '3',
'бор_1':'4',
'бор_2':'5',
'дар_2.1': '7',
'дар_2.3': '6',
 'дар_1': '7',
 'двигатель_1': '8',
 'двигатель_2': '9',
 'дедушка_2': '10',
 'дедушка_1': '11',
 'декрет_2': '12',
 'декрет_1': '13',
 'дерево_1': '14',
 'дерево_2': '15',
 'диалог_1': '16',
 'диалог_2': '17',
#  'диплом_1': '18',
 'диплом_3': '19',
 'диплом_2.1': '20',
 'диплом_2.2': '20',
 'доктор_2': '21',
 'доктор_1': '22',
 'жребий_2': '31',
 'жребий_1': '30',
 'доля_5': '25',
 'доля_1.1': '24',
 'доля_1.2': '24',
 'доля_2.1': '24',
 'доля_2.2': '24',
 'доля_3': '24',
 'доля_4': '24',
 'достижение_2.1': '26',
 'достижение_1.1': '27',
 'достижение_2.2': '26',
 'достижение_1.2': '27',
 'жестокость_1': '28',
 'жестокость_2': '29',
 'затея_2': '32',
 'затея_1': '33',
 'застой_1': '34',
 'застой_2': '35',
 'затишье_2': '36',
 'затишье_1': '37',
 'затишье_3': '37',
 'затмение_2': '38',
 'затмение_1': '39'}

sense_map_russe = {'ЗАМОК_СТРОЕНИЕ.txt': '0',
 'ЗАМОК_ЗАПОР.txt': '1',
 'ЛУК_ОРУЖИЕ.txt': '2',
 'ЛУК_ОВОЩ.txt': '3',
 'БОР_ЭЛЕМЕНТ.txt': '4',
 'БОР_ЛЕС.txt': '5',
 'ДАР_ВРОЖДЕННЫЙ.txt': '6',
 'ДАР_ПОДАРОК.txt': '7',
 'ДВИГАТЕЛЬ_АГРЕГАТ.txt': '8',
 'ДВИГАТЕЛЬ_ДВИЖУЩ_СИЛА.txt': '9',
 'ДЕДУШКА_СТАРИК.txt': '10',
 'ДЕДУШКА_РОДСТВЕНН.txt': '11',
 'ДЕКРЕТ_ОТПУСК.txt': '12',
 'ДЕКРЕТ_ДОКУМЕНТ.txt': '13',
 'ДЕРЕВО_ДЕРЕВЦЕ.txt': '14',
 'ДЕРЕВО_ДРЕВЕСИНА.txt': '15',
 'ДИАЛОГ_БЕСЕДА.txt': '16',
 'ДИАЛОГ_МЖД_СТОРОНАМИ.txt': '17',
 'ДИПЛОМ_ПАМЯТНЫЙ.txt': '19',
 'ДИПЛОМ_РАБОТА.txt': '20',
 'ДОКТОР_НАУК.txt': '21',
 'ДОКТОР_ВРАЧ.txt': '22',
 'ДОЛЯ_ЧАСТЬ.txt': '24',
 'ДОЛЯ_УЧАСТЬ.txt': '25',
 'ДОСТИЖЕНИЕ_ЦЕЛИ.txt': '26',
 'ДОСТИЖЕНИЕ_УРОВНЯ.txt': '27',
 'ЖЕСТОКОСТЬ_БЕСПОЩАДНОСТЬ.txt': '28',
 'ЖЕСТОКОСТЬ_ОБРАЩЕНИЕ.txt': '29',
 'ЖРЕБИЙ_РЕШЕНИЕ.txt': '30',
 'ЖРЕБИЙ_СУДЬБА.txt': '31',
 'ЗАТЕЯ_ЗАБАВА.txt': '32',
 'ЗАТЕЯ_НАЧИНАНИЕ.txt': '33',
 'ЗАСТОЙ_ЗАСТОЙН_ЯВЛЕНИЕ.txt': '34',
 'ЗАСТОЙ_СТАГНАЦИЯ_РАЗВИТ.txt': '35',
 'ЗАТИШЬЕ_СНИЖ_АКТИВНОСТИ.txt': '36',
 'ЗАТИШЬЕ_БЕЗВЕТР_ТИШЬ.txt': '37',
 'ЗАТМЕНИЕ_ОДУРЕНИЕ.txt': '38',
 'ЗАТМЕНИЕ_СВЕТИЛА.txt': '39'}

sense_map_russe_inv = {v : k for k, v in sense_map_russe.items()}

dictionary_txt_map = {k : str(sense_map[sense_map_russe_inv[v]]) for k, v in dictionary_map_russe.items()}

In [ ]:
test_russe_wiki = pd.read_csv(r'C:\Users\russe-wsi-kit\data\main\wiki-wiki\train.csv', sep = '\t')
test_russe_act = pd.read_csv(r'C:\Users\russe-wsi-kit\data\main\active-dict\train.csv', sep='\t')
test_russe_act = test_russe_act.append(test_russe_wiki, ignore_index=True)
stop_list_all_candidates = ['ДИПЛОМ_ВУЗА']
in_train_russe = [i.split('_')[0].lower() for i in sense_map.keys() if i[:-4] not in stop_list_all_candidates]
in_train_russe = list(set(in_train_russe))

In [ ]:
#in case of test dataset RUSSE
absent_meaning = []

root = ET.Element("corpus")
doc = ET.SubElement(root, "document")
par = ET.SubElement(doc, "paragraph")
sent = ET.SubElement(par, "sentence")

for i in range(len(test_russe_act)):
    if i != 48:
        num_s = 0



        if test_russe_act['word'].iloc[i] in in_train_russe:

            full_context = test_russe_act['context'].iloc[i]


            poly_w = test_russe_act['word'].iloc[i]
            dict_sense = test_russe_act['gold_sense_id'].iloc[i]

            if poly_w+'_'+str(dict_sense) in dictionary_txt_map.keys():

                target_sense = dictionary_txt_map[poly_w+'_'+str(dict_sense)]

                tokens = sent_cleaning(full_context)

                for w in tokens:

                    if w == 'лука':
                        w = 'лук'
                    if w == 'даром' and target_sense in ['10', '11', 10, 11]:
                        w = 'дар'
                    if w == 'доле' and target_sense in ['26', '27', 26, 27]:
                        w = 'доля'
                    if w == 'бора' and target_sense in ['4', '5', 4, 5]:
                        w = 'бор'

                    if w !=poly_w:

                        word = ET.SubElement(sent, 'word')
                        word.set('surface_form', w)
                        word.set('lemma', w)

                    else:
                        word = ET.SubElement(sent, 'word')
                        word.set('surface_form', poly_w)
                        word.set('lemma', poly_w)
                        word.set('wn30_key', target_sense)
                        num_s+=1
                if num_s == 0:
                    print(i, poly_w, '\n', full_context, '\n', tokens, target_sense)
                sent = ET.SubElement(par, "sentence")


            else:
                absent_meaning.append(poly_w+'_'+str(dict_sense))


In [ ]:
test_mapping_toloka = {'акция_1':'1', #компании
               'акция_2':'0', #действие
               'байка_1':'3', #ткань
               'байка_2':'2',
               'гвоздика_1':'5',
               'гвоздика_2':'4',
               'гусеница_1':'6', #насекомое
               'гусеница_2':'7', #механизм
               'капот_1':'8', #машинный
               'капот_2':'9', #одежда
               'крона_1':'11',
               'крона_2':'10',
               'рок_1':'13',
               'рок_2':'12',
               'слог_1':'14',
               'слог_2':'15',
               'стопка_1':'16',
               'стопка_2':'17',
               'таз_1':'19',
               'таз_2':'18',
               'такса_1':'20',
               'такса_2':'21'}

sense_map_toloka = {'АКЦИЯ_ДЕЙСТВИЕ.txt': 0,
 'АКЦИЯ_КОМПАНИИ.txt': 1,
 'БАЙКА_ЛОЖЬ.txt': 2,
 'БАЙКА_ТКАНЬ.txt': 3,
 'ГВОЗДИКА_ПРИПРАВА.txt': 4,
 'ГВОЗДИКА_РАСТЕНИЕ.txt': 5,
 'ГУСЕНИЦА_ЛИЧИНКА.txt': 6,
 'ГУСЕНИЦА_МЕХАНИЗМ.txt': 7,
 'КАПОТ_МАШИНЫ.txt': 8,
 'КАПОТ_ОДЕЖДА.txt': 9,
 'КРОНА_ДЕНЬГИ.txt': 10,
 'КРОНА_ДЕРЕВА.txt': 11,
 'РОК_МУЗЫКА.txt': 12,
 'РОК_СУДЬБА.txt': 13,
 'СЛОГ_ЗВУК.txt': 14,
 'СЛОГ_СТИЛЬ.txt': 15,
 'СТОПКА_КУЧА.txt': 16,
 'СТОПКА_ПОСУДА.txt': 17,
 'ТАЗ_КОСТЬ.txt': 18,
 'ТАЗ_ПОСУДА.txt': 19,
 'ТАКСА_ОПЛАТА.txt': 20,
 'ТАКСА_СОБАКА.txt': 21}

sense_map_toloka_inv = {str(v) : k for k, v in sense_map_toloka.items()}

dictionary_txt_map_toloka = {k : str(sense_map[sense_map_toloka_inv[v]]) for k, v in test_mapping_toloka.items()}

In [ ]:
test_2 = pd.read_csv(r'C:\Users\WordSenseRus\WordSenseRus\bts-rnc-crowd.tsv', sep='\t')
in_train_2 = ['акция', 'байка','гвоздика',  'гусеница', 'капот', 'крона','рок','слог','стопка','таз','такса']


In [ ]:

for i in range(len(test_2)):
    
    if test_2['lemma'].iloc[i] in in_train_2:
        
        if pd.isna(test_2['left'].iloc[i]) != True:
            input_left = sent_cleaning(test_2['left'].iloc[i])

            for w in input_left:

                word = ET.SubElement(sent, 'word')
                word.set('surface_form', w)
                word.set('lemma', w)
        
        sense_df = test_2['lemma'].iloc[i]+'_'+str(test_2['sense_id'].iloc[i])
        if sense_df == 'байка_0':
            continue
        if sense_df in dictionary_txt_map_toloka.keys():
            sense = dictionary_txt_map_toloka[sense_df]

            word = ET.SubElement(sent, 'word')
            word.set('surface_form', test_2['lemma'].iloc[i])
            word.set('lemma', test_2['lemma'].iloc[i])
            word.set('wn30_key', sense)

            if pd.isna(test_2['right'].iloc[i]) != True:
                input_right = sent_cleaning(test_2['right'].iloc[i])

                for w in input_right:

                    word = ET.SubElement(sent, 'word')
                    word.set('surface_form', w)
                    word.set('lemma', w)

            sent = ET.SubElement(par, "sentence")

    
tree = ET.ElementTree(root)


In [ ]:
path_to_save_test_xml_file = r"C:\Users\Taiga_1billion\Taiga_1billion\test_taiga_proza_ru_11_03.xml"
xmlstr = minidom.parseString(ET.tostring(root)).toprettyxml(indent="   ")
with open(path_to_save_test_xml_file, "wb") as f:
    f.write(xmlstr.encode('utf-8'))